# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Laboratorio 2: Series de Tiempo

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169

**Curso:** Data Science  
**Sección:** 10  

---

Guatemala, 1 de agosto de 2024


# CONSUMO-2024-05.xlsx  

In [2]:
import pandas as pd

In [4]:

def combineDieselColumns(csvPath):
    print("📂 Iniciando la combinación de las columnas de Diesel...")

    try:
        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(csvPath)
        print(f"✅ Archivo cargado: {csvPath}")
    except Exception as e:
        print(f"❌ Error al cargar {csvPath}: {e}")
        return None
    
    if 'Diesel alto azufre' not in df.columns or 'Diesel bajo azufre' not in df.columns:
        print("❌ Las columnas 'Diesel alto azufre' o 'Diesel bajo azufre' no existen en el archivo CSV.")
        return None

    # Crear la nueva columna 'Diesel'
    df['Diesel'] = df['Diesel alto azufre'].replace(0, pd.NA).combine_first(df['Diesel bajo azufre'].replace(0, pd.NA))
    
    # Eliminar las columnas originales si es necesario
    # df.drop(['Diesel alto azufre', 'Diesel bajo azufre'], axis=1, inplace=True)
    
    print("🏁 Combinación de columnas completada.")
    return df

csvPath = 'Consumo/CONSUMO-2024-05.csv'
dataConsumo = combineDieselColumns(csvPath)

dataConsumo


📂 Iniciando la combinación de las columnas de Diesel...
✅ Archivo cargado: Consumo/CONSUMO-2024-05.csv
🏁 Combinación de columnas completada.


,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Mezclasoleosas,Naftas,PetCoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,Orimulsión,Total,Diesel
0,2000-01-01 00:00:00,0.00,48446.00,296766.99,0.00,0.00,0.00,0.00,194410.476190,1426.00,...,0.00,0.00,0.00,0.00,0.00,0.0,634667.06,0,1.750545e+06,634667.06
1,2000-02-01 00:00:00,0.00,50597.00,328116.44,0.00,0.00,0.00,0.00,174710.552381,1458.00,...,0.00,0.00,0.00,0.00,0.00,0.0,642380.66,0,1.773220e+06,642380.66
2,2000-03-01 00:00:00,0.00,27593.00,368589.63,0.00,0.00,0.00,0.00,189234.066667,1503.00,...,0.00,0.00,0.00,0.00,0.00,0.0,699807.25,0,1.909499e+06,699807.25
3,2000-04-01 00:00:00,0.00,53794.00,396300.47,0.00,0.00,0.00,0.00,174330.607143,1561.00,...,0.00,0.00,0.00,0.00,0.00,0.0,586803.98,0,1.800933e+06,586803.98
4,2000-05-01 00:00:00,0.00,60137.00,449368.92,0.00,0.00,0.00,0.00,191745.147619,1642.00,...,0.00,0.00,0.00,0.00,0.00,0.0,656948.20,0,1.941772e+06,656948.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2024-01-01 00:00:00,28739.00,11541.39,155857.44,-425.40,101931.98,1359012.49,12753.66,548124.450000,1428.57,...,2211.55,63.00,494201.60,91360.15,14764.28,0.0,0.00,0,4.310912e+06,1359012.49
289,2024-02-01 00:00:00,26061.54,16710.19,116497.67,-374.79,90032.19,1340174.42,12428.51,526897.850000,1140.33,...,2061.43,42.15,307639.83,80009.60,21105.25,0.0,0.00,0,4.012227e+06,1340174.42
290,2024-03-01 00:00:00,25159.01,13146.14,243839.73,-124.51,92065.02,1393324.52,12378.90,523990.910000,1510.75,...,1709.22,0.90,534218.36,87066.32,5069.30,0.0,0.00,0,4.476864e+06,1393324.52
291,2024-04-01 00:00:00,30759.23,12942.60,169259.07,-408.66,84078.82,1428143.44,13960.16,531880.190000,1495.49,...,2209.47,49.32,270494.00,76628.15,18204.87,0.0,0.00,0,4.121133e+06,1428143.44


# IMPORTACION-VOLUMEN-2024-05.xlsx  

In [5]:
csvPath = 'Importacion/IMPORTACION-VOLUMEN-2024-05.csv'

dataImportacion = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

dataImportacion

✅ Archivo cargado: Importacion/IMPORTACION-VOLUMEN-2024-05.csv


,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Naftas,Petcoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,MTBE,Orimulsión,Petróleo reconstituido,Total importación
0,2001-01-01 00:00:00,0.00,27748.99,214581.84,0.00,0.00,0.00,0.00,194065.738095,820.00,...,0.00,0.00,0.0,0.00,0.0,566101.99,8402,0.0,715344,2.312639e+06
1,2001-02-01 00:00:00,0.00,7503.57,294609.00,0.00,0.00,0.00,0.00,170703.380952,3054.00,...,0.00,0.00,0.0,0.00,0.0,489525.80,0,0.0,370166,1.769209e+06
2,2001-03-01 00:00:00,0.00,26304.32,315263.80,0.00,0.00,0.00,0.00,161837.371429,677.00,...,0.00,0.00,0.0,0.00,0.0,575559.68,0,0.0,360530,1.945770e+06
3,2001-04-01 00:00:00,0.00,7885.89,205653.00,0.00,0.00,0.00,0.00,163048.642857,3399.00,...,0.00,0.00,0.0,0.00,0.0,437745.42,8184,0.0,359527,1.623638e+06
4,2001-05-01 00:00:00,0.00,8443.16,278371.30,0.00,0.00,0.00,0.00,171518.861905,585.00,...,0.00,0.00,0.0,0.00,0.0,552609.13,12680,0.0,723346,2.262727e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2024-01-01 00:00:00,26693.78,7837.29,99680.22,105.62,101415.50,1409097.15,6710.98,701570.800000,0.00,...,123.60,264918.62,0.0,15383.87,0.0,0.00,0,0.0,0,4.263216e+06
277,2024-02-01 00:00:00,21374.61,5313.11,143951.95,95.09,87352.10,1236861.75,4639.33,916541.700000,5978.37,...,80.00,534021.57,2057.0,24524.03,0.0,0.00,0,0.0,0,4.377134e+06
278,2024-03-01 00:00:00,27993.04,5633.41,163119.06,0.00,105710.45,1477038.00,5007.48,675157.480000,0.00,...,263.28,523701.86,2869.0,3782.28,0.0,0.00,0,0.0,0,4.451150e+06
279,2024-04-01 00:00:00,37444.60,7990.99,77253.65,33.52,85752.17,1294706.12,0.00,473940.660000,3351.23,...,40.00,0.00,7802.0,23555.38,0.0,0.00,0,0.0,0,3.589097e+06


# Precios-Promedio-Nacionales-Diarios-2024-3.xlsx

In [7]:
csvPath = 'PreciosPromedioNacionales/PreciosPromedioNacionales.csv'

dataPrecios = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

dataPrecios

✅ Archivo cargado: PreciosPromedioNacionales/PreciosPromedioNacionales.csv


,FECHA,Tipo de Cambio GTQ/USD,Superior GTQ/GALON,Regular GTQ/GALON,Diesel GTQ/GALON,Bunker GTQ/GALON,Glp Cilindro 25Lbs. GTQ/CIL 25 LB GTQ/CIL 25 LB,Glp Cilindro 25Lbs. GTQ/LB
0,2021-01-01,7.79382,21.91,21.11,17.61,13.40,99.0,3.96
1,2021-01-02,7.79382,21.91,21.11,17.61,13.40,99.0,3.96
2,2021-01-03,7.79382,21.91,21.11,17.61,13.40,99.0,3.96
3,2021-01-04,7.79417,21.91,21.11,17.61,13.45,99.0,3.96
4,2021-01-05,7.79369,21.91,21.11,17.61,13.45,99.0,3.96
...,...,...,...,...,...,...,...,...
1300,2024-07-24,NaN,32.79,31.29,28.09,19.19,110.0,4.40
1301,2024-07-25,NaN,32.79,31.29,28.09,19.19,110.0,4.40
1302,2024-07-26,NaN,32.79,31.29,28.09,19.19,110.0,4.40
1303,2024-07-27,NaN,32.79,31.29,28.09,19.19,110.0,4.40
